In [1]:
import os
from os.path import exists
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

MessageError: ignored

In [ ]:
!pip install transformers
!pip install emoji
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.7 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 42.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 6.9 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=3601b86bd757fc92cdcab85898e2e832536c619f8c92f7d9a41dbd47a30f5879
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [ ]:
# Import libraries
import torch
from transformers import AutoModel, AutoTokenizer, pipeline, AutoModelForSequenceClassification
#from TweetNormalizer import normalizeTweet
import pandas as pd
#import re
# from nltk import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
import numpy as np
#from pysentimiento_master.pysentimiento.preprocessing import preprocess_tweet
#from pysentimiento_master.pysentimiento.preprocessing import *
from tqdm import tqdm
from scipy.special import softmax
import urllib.request
import csv

In [ ]:
# Load in model

model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
# Tryy wit "text-classification"
classifier = pipeline("text-classification", model = model, tokenizer = tokenizer, top_k = 6)

classifier("Oh my god!!!")


In [ ]:
# FUNCION THAT TAKES A DATAFRAME AS INPUT AND OUTPUTS IT WITH LABELS AND SCORES
def get_mood(df):
    
    model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
    tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
 
    classifier = pipeline("text-classification", model = model, tokenizer = tokenizer, return_all_scores=True)
    
    # Take a list of the tweets and find the sentiment and score of each 
    tweets = list(df["text"])
    
    labels = []
    scores = []
    
    for sent in tqdm(tweets):
        
        try:
            results = classifier(sent)
            #Assign label to label
            #df.at[i,"label"] = results[0][0]["label"]
            #df.at[i,"score"] = results[0][0]["score"]
            
            labels.append(results[0][6]["label"])
            scores.append(results[0][6]['score'])
        # Get the exception to print out the error and for which team/time then pass to next month
        except:
        # except Exception as e:
        #     print(e)
            labels.append("DELETE")
            scores.append(100000)
            print("The tweet was: ",sent)
            pass

    # Create an empty col for label and score
    df = df.assign(label = labels)
    df = df.assign(score = scores)
    
    return df

In [ ]:
# Loop through each pre-processed file 
os.chdir("/content/gdrive/My Drive/Diss/2. Analysis")
!ls
working_directory = os.getcwd()
working_directory

 _1_get_vader_scores.py
'1. VADER (working file).ipynb'
'2.1 Roberta Mood.ipynb'
'2.1 Run get_pysent_scores on all data .ipynb'
'2.2 Run get_pysent_scores monthly.ipynb'
'2.5 Pysentimiento v2.ipynb'
 _2_get_pysent_scores_GOOGLE.py
 _2_get_pysent_scores.py
'2. Pysent (working file).ipynb'
 _3_get_prediction_errors.py
'3. Get sentiment scores (working file).ipynb'
 BERTweet-master
 Data
 Merging_team_tweets.ipynb
 OLD
'prediction error (working file).ipynb'
 __pycache__
 pysentimiento_master
'READ ME.txt'
 TweetNormalizer.py


'/content/gdrive/My Drive/Diss/2. Analysis'

In [ ]:
# List of teams and years to iterate through for file names
team_search_queries = {#'giants': [2012],
                      #'devils': [2012],
                      #'knicks': [2012],
                      #'mets': [2012],
                      'yankees': [2012]}

In [ ]:
os.chdir("/content/gdrive/My Drive/Diss/2. Analysis")
working_directory = os.getcwd()

for team, years in team_search_queries.items():
  for year in years:

    # Print out to let me know progess
    print("Currently analysing: {} for {}".format(year, team))

    # Read in the preprocessed csv

    csv_file_name = "/Data/Preprocessed tweets/{}_{}_processed.csv".format(year, team)

    clean_data = pd.read_csv(working_directory + csv_file_name)

    surprise_data = get_mood(clean_data)

    surprise_data.to_csv(working_directory + "/Data/Surprise Roberta Data\{}_{}_surprise.csv".format(year, team))

Currently analysing: 2012 for yankees


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/text_classification.py:90: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  "`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality", UserWarning
100%|██████████| 70542/70542 [1:18:07<00:00, 15.05it/s]
